In [ ]:
# AI-Generated Content & Bias Detection System
# -------------------------------------------------------------

from transformers import pipeline
import re

# -------------------------------------------------------------
# 1. Load Models
# -------------------------------------------------------------
# Zero-shot classification for AI vs Human
ai_detector = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

# Sentiment analysis (English output)
sentiment_model = pipeline("sentiment-analysis")

# Zero-shot classification for harm and bias detection
harm_bias_classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

# -------------------------------------------------------------
# 2. Define Bias Detection
# -------------------------------------------------------------
def detect_bias(text):
    keywords = {
        "Religious Bias": ["شيعة", "سنة", "مسيحي", "يهودي", "إسلام", "يهودية"],
        "Cultural Exclusion": ["غرب", "شرق", "عربي", "أوروبي", "single source", "no diversity"]
    }
    detected = []
    for label, words in keywords.items():
        for w in words:
            if re.search(r"\b" + re.escape(w) + r"\b", text):
                detected.append((label, w))
    return detected

# -------------------------------------------------------------
# 3. Harm & Misinformation Classification
# -------------------------------------------------------------
def classify_harm(text):
    labels = [
        "Misinformation",
        "Cultural Exclusion",
        "Religious Bias",
        "Hate Speech",
        "Sensitive Content",
        "Neutral / Safe"
    ]
    return harm_bias_classifier(text, candidate_labels=labels)

# -------------------------------------------------------------
# 4. Main Analysis
# -------------------------------------------------------------
def analyze_text(text):
    # Step 1: AI vs Human
    ai_result = ai_detector(text, candidate_labels=["AI-generated", "Human-written"])

    # Step 2: Sentiment
    sentiment_result = sentiment_model(text)

    # Step 3: Keyword Bias Detection
    detected_biases = detect_bias(text)

    # Step 4: Harm Level Classification
    harm_result = classify_harm(text)

    # ---------------------------------------------------------
    # Print results in English
    # ---------------------------------------------------------
    print("\n=== AI Content Detection ===")
    print(f"Prediction: {ai_result['labels'][0]} (Confidence: {ai_result['scores'][0]:.2f})")

    print("\n=== Sentiment Analysis ===")
    print(f"Sentiment: {sentiment_result[0]['label']} (Score: {sentiment_result[0]['score']:.2f})")

    print("\n=== Bias Detection (Religious, Cultural & Inclusivity) ===")
    if detected_biases:
        for b in detected_biases:
            print(f"- {b[0]} detected (keyword: {b[1]})")
    else:
        print("- No explicit religious/cultural bias or exclusion detected.")

    print("\n=== Harm Level Classification ===")
    for label, score in zip(harm_result['labels'], harm_result['scores']):
        print(f"- {label}: {score:.2f}")

    print("\n=== Final Assessment ===")
    if detected_biases:
        bias_summary = f"The text contains potential {detected_biases[0][0]} with approx {round(harm_result['scores'][0]*100,1)}% indication."
        print(bias_summary)
    else:
        print("The text seems mostly neutral, culturally inclusive, and does not promote misinformation.")

# -------------------------------------------------------------
# 5. Test Example
# -------------------------------------------------------------
sample_text = """
التاريخ الإسلامي يركز فقط على إنجازات السنة دون ذكر مساهمات الشيعة.
يقدم منظورًا غربيًا فقط ويستبعد التنوع الثقافي.
"""
analyze_text(sample_text)